In [1]:
import { PromptTemplate } from '@langchain/core/prompts'

const greetPrompt = new PromptTemplate({
    template: "Hello, world",
    inputVariables: [],
})

const res = await greetPrompt.format()
res

"Hello, world"

In [3]:

const ParamsPrompt = new PromptTemplate({
    // template: "Hello, {name}! You are {age} years old.", // "Hello, John! You are 18 years old."
    template: "Hello, {name}! You are {{age}} years old.", // "Hello, John! You are {age} years old." 转义
    inputVariables: ["name", "age"],
})

const res1 = await ParamsPrompt.format({ name: "John", age: 18 })
res1

"Hello, John! You are {age} years old."

In [4]:

const autoInferTemplate = PromptTemplate.fromTemplate("good {timeOfDay}, {name}")
autoInferTemplate.inputVariables

const format = await autoInferTemplate.format({ timeOfDay: "morning", name: "John" })
// format

[ "timeOfDay", "name" ]

In [5]:
const initialPrompt = new PromptTemplate({
    template: "这是一个{type}，它是{item}。",
    inputVariables: ["type", "item"],
  });
const partial = await initialPrompt.partial({type: "工具"})

await partial.format({item: "锤子"})
// await partial.format({item: "改锥"})



"这是一个工具，它是锤子。"

In [6]:
// 函数作为变量
const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

const promptWithDate = new PromptTemplate({
  template: "今天是{date}，{activity}。",
  inputVariables: ["date", "activity"],
});

const partialedPromptWithDate = await promptWithDate.partial({
  date: getCurrentDateStr,
});

await partialedPromptWithDate.format({
  activity: "我们去爬山",
});

"今天是8/8/2025，我们去爬山。"

In [7]:
// 函数传参
const getCurrentDateStr = () => {
    return new Date().toLocaleDateString();
  };
  
  function generateGreeting(timeOfDay) {
    return () => {
      const date = getCurrentDateStr()
      switch (timeOfDay) {
        case 'morning':
          return date + ' 早上好';
        case 'afternoon':
          return date + ' 下午好';
        case 'evening':
          return date + ' 晚上好';
        default:
          return date + ' 你好';
      }
    };
  }
  
  const prompt = new PromptTemplate({
    template: "{greeting}!",
    inputVariables: ["greeting"],
  });
  
  const currentTimeOfDay = 'afternoon';
  const partialPrompt = await prompt.partial({
    greeting: generateGreeting(currentTimeOfDay),
  });
  
await partialPrompt.format();

"8/8/2025 下午好!"

In [8]:
import { SystemMessagePromptTemplate } from "@langchain/core/prompts"
import { HumanMessagePromptTemplate } from "@langchain/core/prompts";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const translateInstructionTemplate = SystemMessagePromptTemplate.fromTemplate(`你是一个专
    业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。`);
const userQuestionTemplate = HumanMessagePromptTemplate.fromTemplate("请翻译这句话：{text}")

// const chatPrompt = ChatPromptTemplate.fromMessages([
//     translateInstructionTemplate,
//     userQuestionTemplate,
// ]);
const systemTemplate = "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。";
const humanTemplate = "请翻译这句话：{text}";

const chatPrompt = ChatPromptTemplate.fromMessages([
  ["system", systemTemplate],
  ["human", humanTemplate],
]);
const formattedChatPrompt = await chatPrompt.formatMessages({
    source_lang: "中文",
    target_lang: "法语",
    text: "你好，世界",
});
formattedChatPrompt

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "你是一个专业的翻译员，你的任务是将文本从中文翻译成法语。",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你是一个专业的翻译员，你的任务是将文本从中文翻译成法语。",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "请翻译这句话：你好，世界",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "请翻译这句话：你好，世界",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]

In [ ]:
import {load} from "dotenv"
import {ChatOllama} from "@langchain/community/chat_models/ollama"
import {StringOutputParser} from "@langchain/core/output_parsers"

const env = await load()
const process = {
    env
} 
const chatModel = new ChatOllama({
    model: "qwen3:8b",
    baseUrl: "http://localhost:11434",
  })
const outputParser = new StringOutputParser()

const chain = chatPrompt.pipe(chatModel).pipe(outputParser)

const res = await chain.invoke({
    source_lang: "中文",
    target_lang: "法语",
    text: "你好，世界",
})
process.env

In [ ]:
res

In [ ]:
// PipelinePromptTemplate
import {
    PromptTemplate,
    PipelinePromptTemplate,
  } from "@langchain/core/prompts";
  
  const getCurrentDateStr = () => {
    return new Date().toLocaleDateString();
  };
  
  const fullPrompt = PromptTemplate.fromTemplate(`
  你是一个智能管家，今天是 {date}，你的主人的信息是{info}, 
  根据上下文，完成主人的需求
  {task}`);
  
  const datePrompt = PromptTemplate.fromTemplate("{date}，现在是 {period}")
  const periodPrompt = await datePrompt.partial({
      date: getCurrentDateStr
  })
  
  const infoPrompt =  PromptTemplate.fromTemplate("姓名是 {name}, 性别是 {gender}");
  
  const taskPrompt = PromptTemplate.fromTemplate(`
  我想吃 {period} 的 {food}。 
  再重复一遍我的信息 {info}`);
  
  const composedPrompt = new PipelinePromptTemplate({
    pipelinePrompts: [
      {
        name: "date",
        prompt: periodPrompt,
      },
      {
        name: "info",
        prompt: infoPrompt,
      },
      {
        name: "task",
        prompt: taskPrompt,
      },
    ],
    finalPrompt: fullPrompt,
  });
  
  const formattedPrompt = await composedPrompt.format({
      period: "早上",
      name: "张三",
      gender: "male",
      food: "lemon"
  });
  formattedPrompt

  /**
   * 
   * 一个变量可以多次复用，例如外界输入的 period 在 periodPrompt 和 taskPrompt 都被使用了
pipelinePrompts 中的变量可以被引用，例如我们在 taskPrompt 使用了 infoPrompt 的运行结果
支持动态自定义和 partial。例子中我们也涉及到了这两种特殊的 template
langchain 会自动分析 pipeline 之间的依赖关系，尽可能的进行并行化来提高运行速度
   */